Позаимствовали агрегированные фичи у nazarovo. Спасибо! <br>
https://github.com/nazarovo/data-fusion-2024-competitions2

In [ ]:
# import os
# os.environ['NUMEXPR_MAX_THREADS'] = '32'
# os.environ['NUMEXPR_NUM_THREADS'] = '30'
# os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, CatBoostClassifier, Pool

In [ ]:
train = pd.read_csv('../data/train.csv').set_index("user_id")
train

In [ ]:
report_dates = pd.read_csv('../data/report_dates.csv', parse_dates=['report_dt']).set_index("report")
report_dates["report_dt"] = report_dates["report_dt"].astype('datetime64[ns]')
report_dates

In [ ]:
%%time

clients = pd.read_csv('../data/clients.csv').set_index("user_id")
clients.replace({'employee_count_nm':{'ОТ 101 ДО 500':4,'БОЛЕЕ 1001':6,'ОТ 501 ДО 1000':5,'ДО 10':0,
                                      'ОТ 11 ДО 50':2,'ОТ 51 ДО 100':3,'БОЛЕЕ 500':5,'ОТ 11 ДО 30':2,
                                      'ОТ 31 ДО 50':2}}, inplace=True)
clients = clients.join(report_dates, on="report", how="left")
clients

In [ ]:
%%time
transactions = pd.read_csv('../data/transactions.csv', 
                           parse_dates=['transaction_dttm'], )
                           #low_memory=False, compression='zip')
transactions

In [ ]:
def logsumabs(x):
    return (np.log(x.abs().sum()))
temp_tran = transactions.groupby(['user_id','currency_rk'])['transaction_amt'].agg([logsumabs])
temp_tran = temp_tran.unstack('currency_rk').fillna(0)
temp_tran.columns = ['logsumabs_0','logsumabs_1','logsumabs_2','logsumabs_3']
temp_tran

In [ ]:
%%time
transactions['transaction_dttm'] = transactions['transaction_dttm'].astype('datetime64[ns]')
transactions['dweek'] = transactions['transaction_dttm'].dt.dayofweek# .day//8
transactions['date'] = transactions['transaction_dttm'].dt.date.astype('datetime64[ns]') # .day//8
transactions['sp'] = np.where( transactions['transaction_amt']>0,transactions['transaction_amt'],0)
transactions['sm'] = np.where( transactions['transaction_amt']<0,transactions['transaction_amt'],0)
transactions['ss'] = transactions['sp'] - transactions['sm'] 
transactions

In [ ]:
transactions = transactions.join(clients ,on="user_id", how="left")
transactions['diff_days'] = (transactions['report_dt']-transactions['transaction_dttm']).dt.days
transactions['m'] = (transactions['report_dt'].dt.year-
                     transactions['transaction_dttm'].dt.year)*12+(transactions['report_dt'].dt.month-
                                                                   transactions['transaction_dttm'].dt.month)
transactions

In [ ]:
count_trans = transactions.groupby(['user_id']
    ).agg({
        'transaction_amt':[
            "sum",
            "max",
            "min",
          #  "median",
            "count",
            #"last"
            ],
        "mcc_code":[
            #"last",
            "nunique"
            ],
        "currency_rk":[
            "min",
            "max"
            ],
        'date':[
            "min",
            "max",
            "count"
            ],
        "sp":"sum",
        "sm":"sum",
        "ss":"sum"
        })
count_trans.columns = count_trans.columns.map('_'.join).map(lambda x: "count_trans_"+str(x))
count_trans["count_trans_dates"]=(count_trans["count_trans_date_max"]-count_trans["count_trans_date_min"]).dt.days
count_trans

In [ ]:
# Сумма положительный транзакций
sum_trans_p = transactions[transactions['transaction_amt']>0].groupby(['user_id']
    ).agg({'transaction_amt':["count","mean"]})#.rename(columns={"transaction_amt":"sum_trans_p"})
sum_trans_p.columns = sum_trans_p.columns.map('_'.join).map(lambda x: "sump_"+str(x))
sum_trans_p

In [ ]:
# Сумма отрицательных транзакций
sum_trans_m = transactions[transactions['transaction_amt']<0].groupby(['user_id']
    ).agg({'transaction_amt':["count","mean"]})#.rename(columns={"transaction_amt":"sum_trans_p"})
sum_trans_m.columns = sum_trans_m.columns.map('_'.join).map(lambda x: "summ_"+str(x))
sum_trans_m

In [ ]:
msump = transactions.pivot_table(values="sp",                              
    index="user_id",
    columns="m",
    aggfunc='sum',
    fill_value=0,)
msump.columns = list( map(lambda x: "msump"+str(x), msump.columns ) )
msump

In [ ]:
msumm = transactions.pivot_table(values="sm",
    index="user_id",
    columns="m",
    aggfunc='sum',
    fill_value=0,)
msumm.columns = list( map(lambda x: "msumm"+str(x), msumm.columns ) )
msumm

In [ ]:
msum = transactions.pivot_table(values="transaction_amt",
    index="user_id",
    columns=["m","currency_rk"],
    #aggfunc='sum',
    aggfunc='count',
    fill_value=0,)
msum.columns = list( map(lambda x: "msum"+str(x), msum.columns ) )
msum

In [ ]:
smc = transactions.pivot_table(values="sm",
    index="user_id",
    columns=["m","currency_rk"],
    aggfunc='count',
    fill_value=0,)
smc.columns = list( map(lambda x: "smc"+str(x), smc.columns ) )
smc

In [ ]:
spc = transactions.pivot_table(values="sp",
    index="user_id",
    columns="m",
    aggfunc='count',
    fill_value=0,)
spc.columns = list( map(lambda x: "spc"+str(x), spc.columns ) )
spc

In [ ]:
spdc = transactions.pivot_table(values="sp",
    index="user_id",
    columns="dweek",
    aggfunc='count',
    fill_value=0,)
spdc.columns = list( map(lambda x: "spdc"+str(x), spdc.columns ) )
spdc

spds = transactions.pivot_table(values="sp",
    index="user_id",
    columns="dweek",
    aggfunc='sum',
    fill_value=0,)
spds.columns = list( map(lambda x: "spds"+str(x), spds.columns ) )
spds

In [ ]:
smdc = transactions.pivot_table(values="sm",
    index="user_id",
    columns="dweek",
    aggfunc='count',
    fill_value=0,)
smdc.columns = list( map(lambda x: "smdc"+str(x), smdc.columns ) )
smdc

smds = transactions.pivot_table(values="sm",
    index="user_id",
    columns="dweek",
    aggfunc='sum',
    fill_value=0,)
smds.columns = list( map(lambda x: "smds"+str(x), smds.columns ) )
smds

In [ ]:
max_day_trans = transactions.groupby(['user_id'])[['diff_days']].max().rename(columns={"diff_days":"max_day_trans"})
max_day_trans

In [ ]:
min_day_trans = transactions.groupby(['user_id']
            )[['diff_days']].min().rename(columns={"diff_days":"min_day_trans"})
min_day_trans

In [ ]:
mcc_trans = transactions.pivot_table(    
    values="transaction_amt",
    index="user_id",
    columns="mcc_code",
    aggfunc="sum",
    fill_value=0,)
mcc_trans.columns = list( map(lambda x: "mcc"+str(x), mcc_trans.columns ) )
mcc_trans

In [ ]:
mcc_transc = transactions.pivot_table(    
    values="transaction_amt",
    index="user_id",
    columns="mcc_code",
    aggfunc='count',
    fill_value=0,)
mcc_transc.columns = list( map(lambda x: "mccc"+str(x), mcc_transc.columns ) )
mcc_transc

In [ ]:
mcc_transmc = transactions.pivot_table(    
    values="transaction_amt",
    index="user_id",
    columns=["mcc_code","m"],
    aggfunc=['sum','median'],
    fill_value=0,
)
mcc_transmc.columns = list( map(lambda x: "mccmc"+str(x), mcc_transmc.columns ) )
mcc_transmc

In [ ]:
%%time
date_trans = transactions.groupby("user_id").agg({"date":"unique"}).explode("date").reset_index()#.sort(["user_id","date"])
date_trans["interval"]=date_trans.groupby("user_id")["date"].diff()
date_trans = date_trans.dropna() 
date_trans["interval"]=date_trans["interval"].dt.days
date_trans = date_trans.groupby("user_id").agg({"interval":["last","max"]})
date_trans.columns = list( map(lambda x: "date_"+str(x), date_trans.columns ) )
date_trans

# Итоговая сборка

In [ ]:
df = clients.join(
                count_trans, on="user_id", how="left"
                ).join(temp_tran, on="user_id", how="left"
                ).join(msump, on="user_id", how="left"
                ).join(msumm, on="user_id", how="left"
                ).join(msum, on="user_id", how="left"
                ).join(spc, on="user_id", how="left"
                ).join(smc, on="user_id", how="left"
                ).join(mcc_trans, on="user_id", how="left"
                ).join(mcc_transc, on="user_id", how="left"
                ).join(date_trans, on="user_id", how="left"
                ).join(train, on="user_id", how="left"
                )

df['time'] = df['time'].fillna(-1)
df['time'] = df['time'].astype(np.int32)
df['target'] = df['target'].fillna(-1)
df['target'] = df['target'].astype(np.int8)

df

# train_test_split

In [ ]:
X = df[df['time']!=-1].copy()
y = df[df['time']!=-1][['target']].copy()

test_ids = pd.read_csv('../data/test_ids.csv')
X_train, X_test = X.loc[~X.index.isin(test_ids['user_id'])], X.loc[X.index.isin(test_ids['user_id'])]
y_train, y_test = y.loc[~X.index.isin(test_ids['user_id'])], y.loc[X.index.isin(test_ids['user_id'])]
X_train

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics

In [ ]:
strat_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
frames_for_metamodel = []
metadata = None
cats = []
for i in range(5):
    frames_for_metamodel = []
    for train_index, valid_index in strat_kfold.split(X_train, y_train):
        X_train_, X_val = X_train.drop(['target', 'time'], axis=1).iloc[train_index], X_train.drop(['target', 'time'], axis=1).iloc[valid_index]
        y_train_, y_val = y_train.iloc[train_index], y_train.iloc[valid_index]
        cat = CatBoostClassifier(
            iterations=6500,    
            learning_rate= 0.03, 
            depth=5,
            eval_metric="AUC", 
            random_seed=63+9+i, 
            task_type="GPU",
        )
        cat_features = ["bankemplstatus","customer_age"]
        cat.fit(X_train_,y_train_, eval_set=(X_val, y_val), cat_features=cat_features, verbose=500, plot=True)
        print(i, cat.best_score_)
        cats.append(cat)
        predict = cat.predict_proba(X_val)[:,1]
        frames_for_metamodel.append(pd.DataFrame({'user_id': X_val.index.values, 'pred_agg_0010': predict}))
    if metadata is not None:
        metadata['pred_agg_new'] += pd.concat(frames_for_metamodel, axis=0).reset_index(drop=True)['pred_agg_0010']
    else:
        metadata = pd.concat(frames_for_metamodel, axis=0).reset_index()

In [ ]:
predict_test = np.zeros(X_test.shape[0])
predict_real = np.zeros(df[df['time']==-1][df.columns[:-2]].shape[0])

for cat in cats:
    predict_test += cat.predict_proba(X_test.drop(['target', 'time'], axis=1))[:, 1]
    predict_real += cat.predict_proba(df[df['time']==-1][df.columns[:-2]])[:, 1]

In [ ]:
test_pred = {'user_id': X_test.index.values, 'pred_agg_0010': predict_test}
real_pred = {'user_id': df[df['time']==-1][df.columns[:-2]].index.values, 'pred_agg_0010': predict_real}

test_pred = pd.DataFrame(test_pred)
real_pred = pd.DataFrame(real_pred)

metadata['pred_time_0010'] = metadata['pred_time_0010']/5
test_pred['pred_time_0010'] = test_pred['pred_time_0010']/25
real_pred['pred_time_0010'] = real_pred['pred_time_0010']/25

metadata = pd.concat([metadata, test_pred, real_pred])
metadata.to_csv('../predicts/agg_0010_meta.csv', index=False)

In [ ]:
embeddings_path = f'../embeddings/coles_1.csv'
main_embs = pd.read_csv(embeddings_path)

wtte_embeddings_path = f'../embeddings/wtte_rnn.csv'
wtte_embs = pd.read_csv(wtte_embeddings_path)
main_embs = main_embs.merge(wtte_embs, on='user_id')

In [ ]:
X_train['user_id'] = X_train.index
X_train.reset_index(drop=True, inplace=True)
X_train_emb = X_train.merge(main_embs, on='user_id')

In [ ]:
cat_features = ["bankemplstatus","customer_age"]

In [ ]:
strat_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

X_, y_ = X_train_emb.drop(['time', 'target','report_dt'], axis=1), X_train_emb['time']
time_models = []
for train_index, valid_index in strat_kfold.split(X_train_emb, X_train_emb['target']):
    
    X_train_, X_val = X_.iloc[train_index], X_.iloc[valid_index]
    y_train_, y_val = y_.iloc[train_index], y_.iloc[valid_index]

    model = CatBoostRegressor(
        iterations = 5000,
        depth=5,
        learning_rate=0.02,
        cat_features = cat_features,
        early_stopping_rounds=400,
        random_seed = 42,
        eval_metric='MSLE',
        task_type="GPU",
    )

    model.fit(Pool(X_train_, y_train_, cat_features=cat_features),
              eval_set=Pool(X_val, y_val, cat_features=cat_features),
              verbose=100)
    time_models.append(model)

In [ ]:
scores = []
frames_for_metamodel = []
models = []
for i, (train_index, valid_index) in enumerate(strat_kfold.split(X_train_emb, X_train_emb['target'])):
    train_with_embs_churn = X_train_emb.drop(['time', 'target'], axis=1).copy()
    train_with_embs_churn['new_time'] = time_models[i].predict(X_train_emb)
    X_, y_ = train_with_embs_churn, X_train_emb['target']
    X_train_, X_val = X_.iloc[train_index], X_.iloc[valid_index]
    y_train_, y_val = y_.iloc[train_index], y_.iloc[valid_index]

    model = CatBoostClassifier(
        iterations = 5000,
        depth=4,
        learning_rate=0.005,
        eval_metric='AUC',
        cat_features = cat_features,
        early_stopping_rounds=400,
        task_type="GPU",
    )

    model.fit(Pool(X_train_, y_train_, cat_features=cat_features),
              eval_set=Pool(X_val, y_val, cat_features=cat_features),
              verbose=100)
    models.append(model)
    
    pred = model.predict_proba(X_val)[:, 1]
    frames_for_metamodel.append(pd.DataFrame({'user_id': X_val.user_id.values, 'pred_time_0010': pred}))
    scores.append(metrics.roc_auc_score(y_val, pred))

print(np.mean(scores))
metadata = pd.concat(frames_for_metamodel, axis=0).reset_index(drop=True)

In [ ]:
metadata

In [ ]:
X_test['user_id'] = X_test.index
X_test.reset_index(drop=True, inplace=True)
X_test_emb = X_test.merge(main_embs, on='user_id')

In [ ]:
X_test_ = X_test_emb.copy()
time =[time_models[i].predict(X_test_.drop(['report_dt', 'target', 'time',], axis=1)) for i in range(5)]
X_test_['new_time'] = np.mean(time, axis =0)
predict = np.zeros(X_test_.shape[0])
for i in range(len(models)):
    predict += models[i].predict_proba(X_test_.drop(['target', 'time',], axis=1))[:, 1]
print(metrics.roc_auc_score(X_test_['target'], predict))

In [ ]:
test_pred = {'user_id': X_test_emb.user_id.values, 'pred_time_0010': predict}
test_pred = pd.DataFrame(test_pred)
test_pred['pred_time_0010'] = test_pred['pred_time_0010']/5

In [ ]:
X_ = X.copy()
X_['user_id'] = X_.index
X_.reset_index(drop=True, inplace=True)
X_train_emb = X_.merge(main_embs, on='user_id')

In [ ]:
scores = []
models = []
for i, (train_index, valid_index) in enumerate(strat_kfold.split(X_train_emb, X_train_emb['target'])):
    train_with_embs_churn = X_train_emb.drop(['time', 'target'], axis=1).copy()
    train_with_embs_churn['new_time'] = time_models[i].predict(X_train_emb)
    X_, y_ = train_with_embs_churn, X_train_emb['target']
    X_train_, X_val = X_.iloc[train_index], X_.iloc[valid_index]
    y_train_, y_val = y_.iloc[train_index], y_.iloc[valid_index]

    model = CatBoostClassifier(
        iterations = 5000,
        depth=4,
        learning_rate=0.005,
        eval_metric='AUC',
        cat_features = cat_features,
        early_stopping_rounds=400,
        task_type="GPU",
    )

    model.fit(Pool(X_train_, y_train_, cat_features=cat_features),
              eval_set=Pool(X_val, y_val, cat_features=cat_features),
              verbose=100)
    models.append(model)
    pred = model.predict_proba(X_val)[:, 1]
    scores.append(metrics.roc_auc_score(y_val, pred))

print(np.mean(scores))

In [ ]:
X_real = df[df['time']==-1][df.columns[:-2]].copy()
X_real['user_id'] = X_real.index
X_real.reset_index(drop=True, inplace=True)
X_real_emb = X_real.merge(main_embs, on='user_id')
X_real_ = X_real_emb.copy()

time =[time_models[i].predict(X_real_.drop(['report_dt'], axis=1)) for i in range(5)]
X_real_['new_time'] = np.mean(time, axis =0)
predict = np.zeros(X_real_.shape[0])
for i in range(len(models)):
    predict += models[i].predict_proba(X_real_)[:, 1]

In [ ]:
real_pred = {'user_id': X_real_.user_id.values, 'pred_time_0010': predict}
real_pred = pd.DataFrame(real_pred)
real_pred['pred_time_0010'] = real_pred['pred_time_0010']/5

metadata = pd.concat([metadata, test_pred, real_pred])
metadata['pred_time_0010'] = metadata['pred_time_0010']/5
metadata.to_csv('../predictions/time_0010_pred_meta.csv', index=False)